# Import libraries

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

2023-10-17 19:03:31.671982: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-17 19:03:31.723315: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-17 19:03:31.723364: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-17 19:03:31.723429: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-17 19:03:31.734800: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-17 19:03:31.736668: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

# Import data and split it

In [2]:
df = pd.read_json('../data/test/test_palabras_nuevas.json')
# Delete rows with empty text
df = df[df['PALABRAS_EMPLEO_TEXTO_NUEVAS'] != '']
print(df.shape)
df.head()

(37575, 5)


,ID_OFERTA,ID_PUESTO_ESCO_ULL,CATEGORIA,SUBCATEGORIA,PALABRAS_EMPLEO_TEXTO_NUEVAS
2,cdfb72672340819a6721db72eee187,2825,Comercial y ventas,Comercial,REPARTIDOR REPARTIR
4,8a1bda0c91438bb4133f32e392a1ce,1984,Ventas al detalle,Venta al detalle,DOBLAR LOOK PERCHAR STRADIVARIUS
7,5070f6b07f43d0ac234ceefa6b8595,76,Informatica y telecomunicaciones,Telecomunicaciones,TABLET
8,fe9c5f661a45819a97d8ca343f337c,1508,Informatica y telecomunicaciones,Telecomunicaciones,CONSORCIO MARKETING
9,449a1e78944c59bb38263ab3e37d3f,127,Ingenieros y tecnicos,Otras ingenierias,PRIVILEGIAR


# Delete all single occupations

In [3]:
# Delete all rows with unique 'id_puesto_esco_ull'
df = df[df.duplicated(subset='ID_PUESTO_ESCO_ULL', keep=False)]
# Get the count of each 'id_puesto_esco_ull'
count_df = df.groupby('ID_PUESTO_ESCO_ULL').count()
print(count_df.shape)
num_classes = count_df.shape[0]

(660, 4)


# Set vocabulary size

In [4]:
VOCAB_SIZE = 10458
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(df['PALABRAS_EMPLEO_TEXTO_NUEVAS'].values)

# Show vocab aspect

In [5]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'marketing', 'competencia', 'inmobiliario', 'inicial',
       'emprendedor', 'propiedad', 'acceso', 'inmueble', 'prospeccion',
       'valoracion', 'vender', 'portal', 'espiritu', 'crm', 'comprador',
       'unico', 'propietario', 'publicacion'], dtype='<U22')

In [6]:
# Show an example of the encoding
encoded_example = encoder(df['PALABRAS_EMPLEO_TEXTO_NUEVAS'].values[:20]).numpy()
encoded_example

array([[  35,   30,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [  96,  152,   82,  171,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [  71,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [1862,    2,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [1160,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [   3,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,  

# Create RNN model

In [7]:
# Modify the model architecture to return (None, 766) without time steps
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=3000),
    # Add relu activation function
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compile the model using appropriate loss and metrics for multi-class classification
model.compile(loss='categorical_crossentropy',  # Use categorical crossentropy as your loss function
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

# Print the model summary to review the architecture
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, None)              0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, None, 3000)        31374000  
                                                                 
 dense (Dense)               (None, None, 64)          192064    
                                                                 
 dense_1 (Dense)             (None, None, 660)         42900     
                                                                 
Total params: 31608964 (120.58 MB)
Trainable params: 31608964 (120.58 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Train the model

In [9]:
from tensorflow.keras.utils import to_categorical
# Get the training data from 'descripcion_oferta' and the target from 'id_puesto_esco_ull'
X_train = df['PALABRAS_EMPLEO_TEXTO_NUEVAS'].values
y_train = df['ID_PUESTO_ESCO_ULL'].values

# Step 1: Sort the class labels and assign consecutive integer IDs
sorted_labels = sorted(np.unique(y_train))
class_id_mapping = {class_label: class_id for class_id, class_label in enumerate(sorted_labels)}

# Step 2: Create a dictionary to map class IDs to class labels
class_label_mapping = {class_id: class_label for class_id, class_label in class_id_mapping.items()}

# Step 3: Create functions for conversion
def class_label_to_id(class_label):
    return class_id_mapping.get(class_label)

def class_id_to_label(class_id):
    return class_label_mapping.get(class_id)

# Map class labels to class IDs for y_train
y_train_mapped = np.array([class_label_to_id(label) for label in y_train])

# Step 4: Cast to categorical one-hot encoded variables
# Set num_classes to the number of unique classes
y_train_one_hot = to_categorical(y_train_mapped, num_classes=num_classes)

# Fit the model with one-hot encoded labels (y_train_one_hot)
history = model.fit(X_train, y_train_one_hot, epochs=10)

# Plot the accuracy and loss
import matplotlib.pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.show()

Epoch 1/10


ValueError: in user code:

    File "/home/fulp/clasificador_ofertas_empleo/clasificador-ofertas-empleo-modelo/.venv/lib64/python3.9/site-packages/keras/src/engine/training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "/home/fulp/clasificador_ofertas_empleo/clasificador-ofertas-empleo-modelo/.venv/lib64/python3.9/site-packages/keras/src/engine/training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/fulp/clasificador_ofertas_empleo/clasificador-ofertas-empleo-modelo/.venv/lib64/python3.9/site-packages/keras/src/engine/training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "/home/fulp/clasificador_ofertas_empleo/clasificador-ofertas-empleo-modelo/.venv/lib64/python3.9/site-packages/keras/src/engine/training.py", line 1127, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/fulp/clasificador